In [ ]:
#| default_exp core

# Core

## Imports and utils

In [ ]:
#| export
import json,dateutil,uuid,inspect,types

from fastcore.utils import *
from fastcore.xml import *
from fasthtml.xtend import *

from types import UnionType, SimpleNamespace as ns
from typing import Optional, get_type_hints, get_args, get_origin, Union, Mapping, TypedDict, List
from datetime import datetime
from dataclasses import dataclass,fields,is_dataclass,MISSING,asdict
from collections import namedtuple
from inspect import isfunction,ismethod,signature,Parameter,get_annotations
from functools import wraps, partialmethod
from http import cookies

from fasthtml.starlette import *

empty = Parameter.empty

In [ ]:
import time

from IPython import display
from enum import Enum
from pprint import pprint

from fastcore.test import *
from starlette.testclient import TestClient
from starlette.requests import Headers

In [ ]:
#| export
def is_typeddict(cls:type)->bool:
    "Check if `cls` is a `TypedDict`"
    attrs = 'annotations', 'required_keys', 'optional_keys'
    return isinstance(cls, type) and all(hasattr(cls, f'__{attr}__') for attr in attrs)

In [ ]:
class MyDict(TypedDict): name:str

assert is_typeddict(MyDict)
assert not is_typeddict({'a':1})

In [ ]:
#| export
def is_namedtuple(cls):
    "`True` is `cls` is a namedtuple type"
    return issubclass(cls, tuple) and hasattr(cls, '_fields')

In [ ]:
assert is_namedtuple(namedtuple('tst', ['a']))
assert not is_namedtuple(tuple)

In [ ]:
#| export
def date(s:str):
    "Convert `s` to a datetime"
    return dateutil.parser.parse(s)

In [ ]:
date('2pm')

datetime.datetime(2024, 7, 17, 14, 0)

In [ ]:
#| export
def snake2hyphens(s:str):
    "Convert `s` from snake case to hyphenated and capitalised"
    s = snake2camel(s)
    return camel2words(s, '-')

In [ ]:
snake2hyphens("snake_case")

'Snake-Case'

In [ ]:
#| export
htmx_hdrs = dict(
    boosted="HX-Boosted",
    current_url="HX-Current-URL",
    history_restore_request="HX-History-Restore-Request",
    prompt="HX-Prompt",
    request="HX-Request",
    target="HX-Target",
    trigger_name="HX-Trigger-Name",
    trigger="HX-Trigger")

@dataclass
class HtmxHeaders:
    boosted:str|None=None; current_url:str|None=None; history_restore_request:str|None=None; prompt:str|None=None
    request:str|None=None; target:str|None=None; trigger_name:str|None=None; trigger:str|None=None
    def __bool__(self): return any(hasattr(self,o) for o in htmx_hdrs)

def _get_htmx(h):
    res = {k:h.get(v.lower(), None) for k,v in htmx_hdrs.items()}
    return HtmxHeaders(**res)

In [ ]:
def test_request(url: str='/', headers: dict={}, method: str='get') -> Request:
    scope = {
        'type': 'http',
        'method': method,
        'path': url,
        'headers': Headers(headers).raw,
        'query_string': b'',
        'scheme': 'http',
        'client': ('127.0.0.1', 8000),
        'server': ('127.0.0.1', 8000),
    }
    receive = lambda: {"body": b"", "more_body": False}
    return Request(scope, receive)

In [ ]:
h = test_request(headers=Headers({'HX-Request':'1'}))
_get_htmx(h.headers)

HtmxHeaders(boosted=None, current_url=None, history_restore_request=None, prompt=None, request='1', target=None, trigger_name=None, trigger=None)

In [ ]:
#| export
def str2int(s)->int:
    "Convert `s` to an `int`"
    s = s.lower()
    if s=='on': return 1
    if s=='none': return 0
    return 0 if not s else int(s)

In [ ]:
str2int('1'),str2int('none')

(1, 0)

In [ ]:
def _mk_list(t, v): return [t(o) for o in v]

## Request and response

In [ ]:
#| export
def _fix_anno(t):
    "Create appropriate callable type for casting a `str` to type `t` (or first type in `t` if union)"
    origin = get_origin(t)
    if origin is Union or origin is UnionType or origin in (list,List):
        t = first(o for o in get_args(t) if o!=type(None))
    d = {bool: str2bool, int: str2int}
    res = d.get(t, t)
    if origin in (list,List): res = partial(_mk_list, res)
    return res

In [ ]:
_fix_anno(Union[str,None]),_fix_anno(float)

(str, float)

In [ ]:
_fix_anno(int)('1')

1

In [ ]:
_fix_anno(list[int])(['1','2'])

[1, 2]

In [ ]:
_fix_anno(list[int])('1')

[1]

In [ ]:
#| export
def _form_arg(k, v, d):
    "Get type by accessing key `k` from `d`, and use to cast `v`"
    if v is None: return
    # This is the type we want to cast `v` to
    anno = d.get(k, None)
    if not anno: return v
    return _fix_anno(anno)(v)

In [ ]:
d = dict(k=int, l=List[int])
_form_arg('k', "1", d)

1

In [ ]:
_form_arg('l', "1", d)

[1]

In [ ]:
_form_arg('l', ["1","2"], d)

[1, 2]

In [ ]:
#| export
@dataclass
class HttpHeader: k:str;v:str

In [ ]:
#| export
def _annotations(anno):
    "Same as `get_annotations`, but also works on namedtuples"
    if is_namedtuple(anno): return {o:str for o in anno._fields}
    return get_annotations(anno)

In [ ]:
#| export
def _is_body(anno): return issubclass(anno, (dict,ns)) or _annotations(anno)

In [ ]:
#| export
def _formitem(form, k):
    "Return single item `k` from `form` if len 1, otherwise return list"
    o = form.getlist(k)
    return o[0] if len(o) == 1 else o if o else None

In [ ]:
#| export
def form2dict(form: FormData) -> dict:
    "Convert starlette form data to a dict"
    return {k: _formitem(form, k) for k in form}

In [ ]:
d = [('a',1),('a',2),('b',0)]
fd = FormData(d)
res = form2dict(fd)
test_eq(res['a'], [1,2])
test_eq(res['b'], 0)

In [ ]:
#| export
async def _from_body(req, p):
    form = await req.form()
    anno = p.annotation
    # Get the fields and types of type `anno`, if available
    d = _annotations(anno)
    cargs = {k:_form_arg(k, v, d) for k,v in form2dict(form).items()}
    return anno(**cargs)

In [ ]:
async def f(req):
    def _f(p:HttpHeader): ...
    p = first(signature(_f).parameters.values())
    result = await _from_body(req, p)
    return JSONResponse(result.__dict__)

app = Starlette(routes=[Route('/', f, methods=['POST'])])
client = TestClient(app)

d = dict(k='value1',v=['value2','value3'])
response = client.post('/', data=d)
print(response.json())

{'k': 'value1', 'v': "['value2', 'value3']"}


In [ ]:
#| export
async def _find_p(req, arg:str, p:Parameter):
    "In `req` find param named `arg` of type in `p` (`arg` is ignored for body types)"
    anno = p.annotation
    # If there's an annotation of special types, return object of that type
    if isinstance(anno, type):
        if issubclass(anno, Request): return req
        if issubclass(anno, HtmxHeaders): return _get_htmx(req.headers)
        if issubclass(anno, Starlette): return req.scope['app']
        if _is_body(anno): return await _from_body(req, p)
    # If there's no annotation, check for special names
    if anno is empty:
        if 'request'.startswith(arg.lower()): return req
        if 'session'.startswith(arg.lower()): return req.scope.get('session', {})
        if arg.lower()=='auth': return req.scope.get('auth', None)
        if arg.lower()=='htmx': return _get_htmx(req.headers)
        if arg.lower()=='app': return req.scope['app']
        return None
    # Look through path, cookies, headers, session, query, and body in that order
    res = req.path_params.get(arg, None)
    if res is empty or res is None: res = req.cookies.get(arg, None)
    if res is empty or res is None: res = req.headers.get(snake2hyphens(arg), None)
    if res is empty or res is None: res = nested_idx(req.scope, 'session', arg) or None
    if res is empty or res is None: res = req.query_params.get(arg, None)
    if res is empty or res is None:
        frm = await req.form()
        res = _formitem(frm, arg)
    # Use default param if needed
    if res is empty or res is None: res = p.default
    if res is empty or res is None: raise HTTPException(400, f"Missing required field: {arg}")
    # We can cast str and list[str] to types; otherwise just return what we have
    if not isinstance(res, (list,str)) or anno is empty: return res
    anno = _fix_anno(anno)
    try: return [anno(o) for o in res] if isinstance(res,list) else anno(res)
    except ValueError: raise HTTPException(404, req.url.path) from None

async def _wrap_req(req, params):
    return [await _find_p(req, arg, p) for arg,p in params.items()]

In [ ]:
def g(req, this:Starlette, a:str, b:HttpHeader): ...

async def f(req):
    a = await _wrap_req(req, signature(g).parameters)
    return Response(str(a))

app = Starlette(routes=[Route('/', f, methods=['POST'])])
client = TestClient(app)

response = client.post('/?a=1', data=d)
print(response.text)

[<starlette.requests.Request object>, <starlette.applications.Starlette object>, '1', HttpHeader(k='value1', v="['value2', 'value3']")]


In [ ]:
#| export
def flat_xt(lst):
    "Flatten lists, except for `XT`s"
    result = []
    for item in lst:
        if isinstance(item, (list,tuple)) and not isinstance(item, XT): result.extend(item)
        else: result.append(item)
    return result

In [ ]:
x = XT('a',1)
flat_xt([x, x, [x,x]])

[['a', 1, {}], ['a', 1, {}], ['a', 1, {}], ['a', 1, {}]]

In [ ]:
#| export
def _xt_resp(req, resp, hdrs, **bodykw):
    if not isinstance(resp, tuple): resp = (resp,)
    resp = resp + tuple(req.injects)
    http_hdrs,resp = partition(resp, risinstance(HttpHeader))
    http_hdrs = {o.k:str(o.v) for o in http_hdrs}
    titles,bdy = partition(resp, lambda o: getattr(o, 'tag', '') in ('title','meta'))
    if resp and 'hx-request' not in req.headers and not any(getattr(o, 'tag', '')=='html' for o in resp):
        if not titles: titles = [Title('FastHTML page')]
        resp = Html(Head(*titles, *flat_xt(hdrs)), Body(bdy, **bodykw))
    return HTMLResponse(to_xml(resp), headers=http_hdrs)

In [ ]:
#| export
def _wrap_resp(req, resp, cls, hdrs, **bodykw):
    if not resp: resp=()
    if isinstance(resp, FileResponse) and not os.path.exists(resp.path): raise HTTPException(404, resp.path)
    if isinstance(resp, Response): return resp
    if cls is not empty: return cls(resp)
    if isinstance(resp, (list,tuple,HttpHeader)) or hasattr(resp, '__xt__'): return _xt_resp(req, resp, hdrs, **bodykw)
    if isinstance(resp, str): cls = HTMLResponse
    elif isinstance(resp, Mapping): cls = JSONResponse
    else:
        resp = str(resp)
        cls = HTMLResponse
    return cls(resp)

In [ ]:
#| export
class Beforeware:
    def __init__(self, f, skip=None): self.f,self.skip = f,skip or []

In [ ]:
#| export
def _wrap_ep(f, hdrs, before, after, **bodykw):
    if not (isfunction(f) or ismethod(f)): return f
    sig = signature(f)
    params = sig.parameters
    cls = sig.return_annotation

    async def _f(req):
        resp = None
        req.injects = []
        for b in before:
            if not resp:
                if isinstance(b, Beforeware): bf,skip = b.f,b.skip
                else: bf,skip = b,[]
                if not any(re.match(r, req.url.path) for r in skip):
                    wreq = await _wrap_req(req, signature(bf).parameters)
                    resp = bf(*wreq)
                    if is_async_callable(bf): resp = await resp
        if not resp:
            wreq = await _wrap_req(req, params)
            resp = f(*wreq)                            
            if is_async_callable(f): resp = await resp
        for a in after:
            _,*wreq = await _wrap_req(req, signature(a).parameters)
            nr = a(resp, *wreq)
            if nr: resp = nr
        return _wrap_resp(req, resp, cls, hdrs, **bodykw)
    return _f

## Websockets

In [ ]:
#| export
def _find_wsp(ws, data, hdrs, arg:str, p:Parameter):
    "In `data` find param named `arg` of type in `p` (`arg` is ignored for body types)"
    anno = p.annotation
    if isinstance(anno, type):
        if issubclass(anno, HtmxHeaders): return _get_htmx(hdrs)
        if issubclass(anno, Starlette): return ws.scope['app']
    if anno is empty:
        if arg.lower()=='ws': return ws
        if arg.lower()=='data': return data
        if arg.lower()=='htmx': return _get_htmx(hdrs)
        if arg.lower()=='app': return ws.scope['app']
        if arg.lower()=='send': return partial(_send_ws, ws)
        return None
    res = data.get(arg, None)
    if res is empty or res is None: res = headers.get(snake2hyphens(arg), None)
    if res is empty or res is None: res = p.default
    # We can cast str and list[str] to types; otherwise just return what we have
    if not isinstance(res, (list,str)) or anno is empty: return res
    anno = _fix_anno(anno)
    return [anno(o) for o in res] if isinstance(res,list) else anno(res)

def _wrap_ws(ws, data, params):
    hdrs = data.pop('HEADERS', {})
    return [_find_wsp(ws, data, hdrs, arg, p) for arg,p in params.items()]

In [ ]:
#| export
async def _send_ws(ws, resp):
    if not resp: return
    res = to_xml(resp) if isinstance(resp, (list,tuple)) or hasattr(resp, '__xt__') else resp
    await ws.send_text(res)

def _ws_endp(recv, conn=None, disconn=None, hdrs=None, before=None, **bodykw):
    cls = type('WS_Endp', (WebSocketEndpoint,), {"encoding":"text"})
    
    async def _generic_handler(handler, ws, data=None):
        wd = _wrap_ws(ws, loads(data) if data else {}, signature(handler).parameters)
        resp = handler(*wd)
        if resp:
            if is_async_callable(handler): resp = await resp
            await _send_ws(ws, resp)

    async def _connect(self, ws):
        await ws.accept()
        await _generic_handler(conn, ws)
    async def _disconnect(self, ws, close_code): await _generic_handler(disconn, ws)
    async def _recv(self, ws, data): await _generic_handler(recv, ws, data)

    if    conn: cls.on_connect    = _connect
    if disconn: cls.on_disconnect = _disconnect
    cls.on_receive = _recv
    return cls

In [ ]:
def on_receive(self, msg:str): return f"Message text was: {msg}"
c = _ws_endp(on_receive)
app = Starlette(routes=[WebSocketRoute('/', _ws_endp(on_receive))])

cli = TestClient(app)
with cli.websocket_connect('/') as ws:
    ws.send_text('{"msg":"Hi!"}')
    data = ws.receive_text()
    assert data == 'Message text was: Hi!'

## Routing and application

In [ ]:
#| export
class WS_RouteX(WebSocketRoute):
    def __init__(self, path:str, recv, conn:callable=None, disconn:callable=None, *,
                 name=None, middleware=None, hdrs=None, before=None, **bodykw):
        super().__init__(path, _ws_endp(recv, conn, disconn, hdrs, before, **bodykw), name=name, middleware=middleware)

In [ ]:
#| export
class RouteX(Route):
    def __init__(self, path:str, endpoint, *, methods=None, name=None, include_in_schema=True, middleware=None,
                hdrs=None, before=None, after=None, **bodykw):
        ep = _wrap_ep(endpoint, hdrs, before=before, after=after, **bodykw)
        super().__init__(path, ep, methods=methods, name=name, include_in_schema=include_in_schema, middleware=middleware)

In [ ]:
#| export
class RouterX(Router):
    def __init__(self, routes=None, redirect_slashes=True, default=None, on_startup=None, on_shutdown=None,
                 lifespan=None, *, middleware=None, hdrs=None, before=None, after=None, **bodykw):
        super().__init__(routes, redirect_slashes, default, on_startup, on_shutdown,
                 lifespan=lifespan, middleware=middleware)
        self.hdrs,self.bodykw,self.before,self.after = hdrs,bodykw,before,after

    def add_route( self, path: str, endpoint: callable, methods=None, name=None, include_in_schema=True):
        route = RouteX(path, endpoint=endpoint, methods=methods, name=name, include_in_schema=include_in_schema,
                       hdrs=self.hdrs, before=self.before, after=self.after, **self.bodykw)
        self.routes.append(route)

    def add_ws( self, path: str, recv: callable, conn:callable=None, disconn:callable=None, name=None):
        route = WS_RouteX(path, recv=recv, conn=conn, disconn=disconn, name=name, hdrs=self.hdrs, before=self.before, **self.bodykw)
        self.routes.append(route)

In [ ]:
#| export
htmxscr   = Script(src="https://unpkg.com/htmx.org@next/dist/htmx.min.js")
htmxwsscr = Script(src="https://unpkg.com/htmx-ext-ws/ws.js")
surrsrc   = Script(src="https://cdn.jsdelivr.net/gh/answerdotai/surreal@1.3.0/surreal.js")
scopesrc  = Script(src="https://cdn.jsdelivr.net/gh/gnat/css-scope-inline@main/script.js")
viewport  = Meta(name="viewport", content="width=device-width, initial-scale=1, viewport-fit=cover")
charset   = Meta(charset="utf-8")

In [ ]:
#| export
def get_key(key=None, fname='.sesskey'):
    if key: return key
    fname = Path(fname)
    if fname.exists(): return fname.read_text()
    key = str(uuid.uuid4())
    fname.write_text(key)
    return key

In [ ]:
get_key()

'2c25d03b-24f2-4946-833a-50ac832a1576'

In [ ]:
#| export
def _list(o): return [] if not o else list(o) if isinstance(o, (tuple,list)) else [o]

In [ ]:
#| export
class FastHTML(Starlette):
    def __init__(self, debug=False, routes=None, middleware=None, exception_handlers=None,
                 on_startup=None, on_shutdown=None, lifespan=None, hdrs=None, before=None, after=None, default_hdrs=True,
                 secret_key=None, session_cookie='session_', max_age=365*24*3600, ws_hdr=False, sess_path='/',
                 same_site='lax', sess_https_only=False, sess_domain=None, key_fname='.sesskey', **bodykw):
        middleware,before,after = _list(middleware),_list(before),_list(after)
        secret_key = get_key(secret_key, key_fname)
        sess = Middleware(SessionMiddleware, secret_key=secret_key, session_cookie=session_cookie,
                          max_age=max_age, path=sess_path, same_site=same_site,
                          https_only=sess_https_only, domain=sess_domain)
        middleware.append(sess)
        super().__init__(debug, routes, middleware, exception_handlers, on_startup, on_shutdown, lifespan=lifespan)
        hdrs = list([] if hdrs is None else hdrs)
        if default_hdrs: hdrs = [charset, viewport, htmxscr,surrsrc,scopesrc] + hdrs
        if ws_hdr: hdrs.append(htmxwsscr)
        self.router = RouterX(routes, on_startup=on_startup, on_shutdown=on_shutdown, lifespan=lifespan, hdrs=hdrs,
                              before=before, after=after, **bodykw)

    def route(self, path:str, methods=None, name=None, include_in_schema=True):
        def f(func):
            m = [methods] if isinstance(methods,str) else [func.__name__] if not methods else methods
            self.router.add_route(path, func, methods=m, name=name, include_in_schema=include_in_schema)
            return func
        return f

    def ws(self, path:str, conn=None, disconn=None, name=None):
        def f(func):
            self.router.add_ws(path, func, conn=conn, disconn=disconn, name=name)
            return func
        return f

all_meths = 'get post put delete patch head trace options'.split()
for o in all_meths: setattr(FastHTML, o, partialmethod(FastHTML.route, methods=o))

## Extras

In [ ]:
#| export
def cookie(key: str, value="", max_age=None, expires=None, path="/", domain=None, secure=False, httponly=False, samesite="lax",):
    "Create a 'set-cookie' `HttpHeader`"
    cookie = cookies.SimpleCookie()
    cookie[key] = value
    if max_age is not None: cookie[key]["max-age"] = max_age
    if expires is not None:
        cookie[key]["expires"] = format_datetime(expires, usegmt=True) if isinstance(expires, datetime) else expires
    if path is not None: cookie[key]["path"] = path
    if domain is not None: cookie[key]["domain"] = domain
    if secure: cookie[key]["secure"] = True
    if httponly: cookie[key]["httponly"] = True
    if samesite is not None:
        assert samesite.lower() in [ "strict", "lax", "none", ], "must be 'strict', 'lax' or 'none'"
        cookie[key]["samesite"] = samesite
    cookie_val = cookie.output(header="").strip()
    return HttpHeader("set-cookie", cookie_val)

In [ ]:
#| export
def reg_re_param(m, s):
    cls = get_class(f'{m}Conv', sup=StringConvertor, regex=s)
    register_url_convertor(m, cls())

In [ ]:
#| export
# Starlette doesn't have the '?', so it chomps the whole remaining URL
reg_re_param("path", ".*?")
reg_re_param("static", "ico|gif|jpg|jpeg|webm|css|js|woff|png|svg|mp4|webp|ttf|otf|eot|woff2|txt|xml|html")

In [ ]:
#| export
class MiddlewareBase:
    async def __call__(self, scope, receive, send) -> None:
        if scope["type"] not in ["http", "websocket"]:
            await self.app(scope, receive, send)
            return
        return HTTPConnection(scope)

## Tests

In [ ]:
def get_cli(app): return app,TestClient(app),app.route

In [ ]:
app,cli,rt = get_cli(FastHTML(secret_key='soopersecret'))

In [ ]:
@rt("/hi")
def get(): return 'Hi there'

r = cli.get('/hi')
r.text

'Hi there'

In [ ]:
@rt("/hi")
def post(): return 'Postal'

cli.post('/hi').text

'Postal'

In [ ]:
@app.get("/")
def show_host(req): return req.headers['host']

cli.get('/').text

'testserver'

In [ ]:
@rt('/user/{nm}', name='gday')
def get(nm:str): return f"Good day to you, {nm}!"

cli.get('/user/Alexis').text

'Good day to you, Alexis!'

In [ ]:
test_eq(app.router.url_path_for('gday', nm='Jeremy'), '/user/Jeremy')

In [ ]:
hxhdr = {'headers':{'hx-request':"1"}}

@rt('/xt')
def get(): return Title('Foo'),H1('bar')

txt = cli.get('/xt').text
assert '<title>Foo</title>' in txt and '<h1>bar</h1>' in txt and '<html>' in txt

@rt('/xt2')
def get(): return H1('bar')

txt = cli.get('/xt2').text
assert '<title>FastHTML page</title>' in txt and '<h1>bar</h1>' in txt and '<html>' in txt

assert cli.get('/xt2', **hxhdr).text.strip() == '<h1>bar</h1>'

@rt('/xt3')
def get(): return Html(Head(Title('hi')), Body(P('there')))

txt = cli.get('/xt3').text
assert '<title>FastHTML page</title>' not in txt and '<title>hi</title>' in txt and '<p>there</p>' in txt

In [ ]:
def test_r(cli, path, exp, meth='get', hx=False, **kwargs):
    if hx: kwargs['headers'] = {'hx-request':"1"}
    test_eq(getattr(cli, meth)(path, **kwargs).text, exp)

app.chk = 'foo'
ModelName = str_enum('ModelName', "alexnet", "resnet", "lenet")
fake_db = [{"name": "Foo"}, {"name": "Bar"}]

In [ ]:
@rt('/html/{idx}')
async def get(idx:int): return Body(H4(f'Next is {idx+1}.'))

reg_re_param("imgext", "ico|gif|jpg|jpeg|webm")

@rt(r'/static/{path:path}{fn}.{ext:imgext}')
def get(fn:str, path:str, ext:str): return f"Getting {fn}.{ext} from /{path}"

@rt("/models/{nm}")
def get(nm:ModelName): return nm

@rt("/files/{path}")
async def get(path: Path): return path.with_suffix('.txt')

@rt("/items/")
def get(idx:int|None = 0): return fake_db[idx]

In [ ]:
test_r(cli, '/html/1', '<body>\n  <h4>Next is 2.</h4>\n</body>\n', hx=True)
test_r(cli, '/static/foo/jph.ico', 'Getting jph.ico from /foo/')
test_r(cli, '/models/alexnet', 'alexnet')
test_r(cli, '/files/foo', 'foo.txt')
test_r(cli, '/items/?idx=1', '{"name":"Bar"}')
test_r(cli, '/items/', '{"name":"Foo"}')
assert cli.get('/items/?idx=g').status_code==404

In [ ]:
@app.get("/booly/")
def _(coming:bool=True): return 'Coming' if coming else 'Not coming'

@app.get("/datie/")
def _(d:date): return d

@app.get("/ua")
async def _(user_agent:str): return user_agent

@app.get("/hxtest")
def _(htmx): return htmx.request

@app.get("/hxtest2")
def _(foo:HtmxHeaders, req): return foo.request

@app.get("/app")
def _(app): return app.chk

@app.get("/app2")
def _(foo:FastHTML): return foo.chk,HttpHeader("mykey", "myval")

In [ ]:
test_r(cli, '/booly/?coming=true', 'Coming')
test_r(cli, '/booly/?coming=no', 'Not coming')
date_str = "17th of May, 2024, 2p"
test_r(cli, f'/datie/?d={date_str}', '2024-05-17 14:00:00')
test_r(cli, '/ua', 'FastHTML', headers={'User-Agent':'FastHTML'})
test_r(cli, '/hxtest' , '1', headers={'HX-Request':'1'})
test_r(cli, '/hxtest2', '1', headers={'HX-Request':'1'})
test_r(cli, '/app' , 'foo')

In [ ]:
r = cli.get('/app2', **hxhdr)
test_eq(r.text, 'foo\n')
test_eq(r.headers['mykey'], 'myval')

In [ ]:
@app.post('/profile/me')
def profile_update(username: str): return username

In [ ]:
# Working post request
test_r(cli, '/profile/me', 'Alexis', 'post',
       data={'username' : 'Alexis'})

In [ ]:
# Failing post request due to missing required parameter
try:
    test_r(cli, '/profile/me', 'Alexis', 'post', data={})
except AssertionError as e:
    assert "Missing required field: username" in str(e)

In [ ]:
@app.post('/pet/dog')
def pet_dog(dogname: str = None): return dogname

# Working post request with optional parameter
test_r(cli, '/pet/dog', 'None', 'post', data={})

AssertionError: ==:
Missing required field: dogname
None

In [ ]:
@dataclass
class Bodie: a:int;b:str

@rt("/bodie/{nm}/")
async def post(nm:str, data:Bodie):
    res = asdict(data)
    res['nm'] = nm
    return res

@app.post("/bodied/")
async def bodied(nm:str, data:dict): return data

nt = namedtuple('Bodient', ['a','b'])

@app.post("/bodient/")
async def bodient(data:nt): return data._asdict()

class BodieTD(TypedDict): a:int;b:str='foo'

@app.post("/bodietd/")
async def bodient(data:BodieTD): return data

class Bodie2:
    a:int|None; b:str
    def __init__(self, a, b='foo'): store_attr()

@app.post("/bodie2/")
async def bodie(d:Bodie2): return f"a: {d.a}; b: {d.b}"

In [ ]:
d = dict(a=1, b='foo')

test_r(cli, '/bodie/me', '{"a":1,"b":"foo","nm":"me"}', 'post', data=d)
# This test should not be working, as the "nm" arguement isn't provided and there isn't a default
# test_r(cli, '/bodied/', '{"a":"1","b":"foo"}', 'post', data=d)
test_r(cli, '/bodie2/', 'a: 1; b: foo', 'post', data={'a':1})
test_r(cli, '/bodient/', '{"a":"1","b":"foo"}', 'post', data=d)
test_r(cli, '/bodietd/', '{"a":1,"b":"foo"}', 'post', data=d)

In [ ]:
@rt("/setcookie")
def get(req): return cookie('now', datetime.now())

@rt("/getcookie")
async def get(now:date): return f'Cookie was set at time {now.time()}'

In [ ]:
print(cli.get('/setcookie').text)
time.sleep(0.01)
cli.get('/getcookie').text

In [ ]:
@rt("/setsess")
async def get(sess):
    now = datetime.now()
    sess['noo'] = str(now)
    return f'Set to {now}'

@rt("/getsess")
async def get(noo:date): return f'Session time: {noo.time()}'

In [ ]:
print(cli.get('/setsess').text)
time.sleep(0.01)
cli.get('/getsess').text

In [ ]:
@rt("/upload")
async def post(uploadfile:str): return (await uploadfile.read()).decode()

fn = '../CHANGELOG.md'
data = {'message': 'Hello, world!'}
with open(fn, 'rb') as f:
    print(cli.post('/upload', files={'uploadfile': f}, data=data).text[:80])

In [ ]:
from fasthtml.authmw import user_pwd_auth

In [ ]:
auth = user_pwd_auth(testuser='spycraft')
app,cli,rt = get_cli(FastHTML(middleware=[auth]))

@rt("/locked")
def get(auth): return 'Hello, ' + auth

test_eq(cli.get('/locked').text, 'not authenticated')
test_eq(cli.get('/locked', auth=("testuser","spycraft")).text, 'Hello, testuser')

In [ ]:
hdrs, routes = app.router.hdrs, app.routes

In [ ]:
from fasthtml.live_reload import FastHTMLWithLiveReload

In [ ]:
app,cli,rt = get_cli(FastHTMLWithLiveReload())

@rt("/hi")
def get(): return 'Hi there'

test_eq(cli.get('/hi').text, "Hi there")

lr_hdrs, lr_routes = app.router.hdrs, app.routes
test_eq(len(lr_hdrs), len(hdrs)+1)
assert app.LIVE_RELOAD_HEADER in lr_hdrs
test_eq(len(lr_routes), len(routes)+1)
assert app.LIVE_RELOAD_ROUTE in lr_routes

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()